In [4]:
"""
JSON output basically "from dctap", but tweaked.
"""
ap = Path('example02.json').read_text(encoding="UTF-8")
for line in ap.splitlines():
   print(line)

{
    "namespaces": {
        "ex": "http://ex.example/#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "school": "http://school.example/#",
        "xsd": "http://www.w3.org/2001/XMLSchema#"
    },
    "shapes": [
        {
        "shapeID": "school:Enrollee",
        "statement_templates": [
                {
                    "max": "2",
                    "min": "1",
                    "propertyID": "ex:hasGuardian",
                    "valueNodeType": "iri"
                },
                {
                    "MaxInclusive": "20",
                    "MinInclusive": "13",
                    "propertyID": "foaf:age",
                    "valueDataType": "xsd:integer"
                }
            ]
        }
    ]
}


In [7]:
"""
ShEx template in Jinja2.
"""
shex_jinja = Path('example02.jinja').read_text(encoding="UTF-8")
for line in shex_jinja.splitlines():
   print(line)

{%- for prefix, uri in namespaces.items() %}
PREFIX {{prefix}}: <{{uri}}>
{%- endfor %}
BASE <http://purl.org/yama/example/mybook/v021/#>

{%- for shape in shapes %}
{{shape.shapeID}} {
  {%- for statement in shape. statement_templates %}
  {{statement.property}} {{statement.propertyID}} 
  {%- if statement.valueNodeType %} {{statement.valueNodeType|upper}}{% endif -%}
  {%- if statement.valueDataType %} {{statement.valueDataType}}{% endif -%}
  {%- if statement.min or statement.max %} { {{-statement.min-}}
        {%- if statement.max -%}
         ,{{statement.max}} 
        {%- endif -%} }
  {%- endif -%}
  {%- if statement.MaxInclusive %} MaxInclusive {{statement.MaxInclusive}} {%- endif -%}
  {%- if statement.MinInclusive %} MinInclusive {{statement.MinInclusive}} {%- endif -%}
  {%- if not loop.last -%} ;{%- endif -%}
  {%- endfor %}
}
{%- endfor %}


In [3]:
from jinja2 import Template
template = Template(shex_jinja)
shexc_output = template.render(ap)
print(shexc_output)

ValueError: dictionary update sequence element #0 has length 1; 2 is required